<a href="https://colab.research.google.com/github/Nohgyumin0812/Sector_Momentum/blob/main/sector_momentum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import time
import requests
import pandas as pd
import requests as rq
from io import BytesIO
import pandas as pd
import matplotlib.pyplot as plt

path = '/content/drive/MyDrive'
from google.colab import drive
drive.mount('/content/drive')

In [3]:
# 섹터
# 자동차, 반도체, 헬스케어, 은행, 에너지화학, 철강, 방송통신, 건설, 증권, 기계장비, 보험, 운송, 경기소비재, 필수소비재, 미디어엔터, 정보기술, 유틸, 음식료, 유통,

In [14]:
sector_code = {'KRX 자동차' : ["5", "043"],
'KRX 반도체' : ["5", "044"],
'KRX 헬스케어' : ["5", "045"],
'KRX 은행' : ["5", "046"],
'KRX 에너지화학' : ['5', "048"],
'KRX 철강' : ["5", '049'],
"KRX 방송통신" :['5', "051"],
"KRX 건설" :["5", "052"],
"KRX 증권" : ["5", "054"],
"KRX 기계장비" : ["5", "055"],
"KRX 보험" : ["5", "056"],
"KRX 운송" : ["5", "057"],
"KRX 경기소비재" : ["5", "061"],
"KRX 필수소비재" : ["5", "062"],
"KRX 미디어엔터" : ["5", "063"],
"KRX 정보기술" : ["5", "064"],
"KRX 유틸리티" : ["5", "065"],
"음식료품" :["1", "005"],
"유통업" :["1","016"]
}

mo1_start_date = '20231214'
mo1_end_date = '20240114'

mo3_start_date = '20231014'
mo3_end_date = '20240114'


df_list = []
ret_lst = []
sector_name_lst = list(sector_code.keys())

In [34]:
# 섹터 이름
# generate.cmd에서 Request URL과 동일

def sector_return(strtDd, endDd):
   ret_lst = []
   for i in range(len(sector_name_lst)):
      gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'

      tboxindIdx_finder_equidx0_1 = sector_name_lst[i]
      indIdx = list(sector_code.values())[i][0]
      indIdx2 = list(sector_code.values())[i][1]
# generate.cmd에서 Form Data와 동일
      gen_otp_data = {'locale': 'ko_KR',
'tboxindIdx_finder_equidx0_1': tboxindIdx_finder_equidx0_1, # KRX 자동차
'indIdx': indIdx, # 5
'indIdx2': indIdx2, # 043
'codeNmindIdx_finder_equidx0_1': tboxindIdx_finder_equidx0_1, # KRX 자동차
'param1indIdx_finder_equidx0_1': "",
'strtDd': strtDd, # 시작날짜(ex. 20230324)
'endDd': endDd, # 끝날짜(ex. 20230401)
'share': '2',
'money': '3',
'csvxls_isNo': 'false',
'name': 'fileDown',
'url': 'dbms/MDC/STAT/standard/MDCSTAT00301'
}
# 헤더 부분에 리퍼러(Referer)를 추가
# 리퍼러란 링크를 통해서 각각의 웹사이트로 방문할 때 남는 흔적

      headers = {'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}
      otp = rq.post(gen_otp_url, gen_otp_data, headers=headers).text

#download.cmd 에서 General의 Request URL 부분
      down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'

# requests Module의 post함수를 이용하여 해당 url에 접속하여 otp코드를 제출
      down_sector  = rq.post(down_url, {'code':otp}, headers=headers)

# 다운 받은 csv파일을 pandas의 read_csv 함수를 이용하여 읽음
# read_csv 함수의 argument에 적합할 수 있도록 BytesIO함수를 이용하여 바이너 스트림 형태로
      sector = pd.read_csv(BytesIO(down_sector.content), encoding='EUC-KR')

      sector_name = list(sector_code.keys())[i]
      ret = (sector.iloc[0, 1] - sector.iloc[-1, 1])/ sector.iloc[-1, 1]
      ret_lst.append(ret)
      df = pd.DataFrame(zip(sector_name_lst, ret_lst))
      df.columns = ['sector_name', f'ret_{strtDd}']
      df = df.sort_values(by = f'ret_{strtDd}', ascending = False).reset_index(drop = True)

   return df




df_mo1 = sector_return(mo1_start_date, mo1_end_date)
df_mo3 = sector_return(mo3_start_date, mo3_end_date)
df = pd.merge(df_mo1, df_mo3).rename(columns = {'ret_20231214' : 'mo1', 'ret_20231014' : 'mo3'})
print(df)
